In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import options
import selenium
import json
from datetime import datetime
import time

SERIE_A = "https://oddspedia.com/br/futebol/brasil/brasileirao-serie-a"
SERIE_B = "https://oddspedia.com/br/futebol/brasil/brasileirao-serie-b"
LIBERTADORES = "https://oddspedia.com/br/futebol/america-do-sul/conmebol-libertadores"
API_BASE = "https://oddspedia.com/api/v1/getMatchOdds?wettsteuer=0&geoCode=BR&bookmakerGeoCode=BR&bookmakerGeoState=&language=br" 
# &matchid={matchid}&oddGroupID={odds_market}&inplay=0
# odds_market = {1, 4}; 1 = h2h; 4 = over/under

In [7]:
opt = options.Options()
opt.add_argument("--start-maximized")
driver = webdriver.Chrome(options=opt)

In [8]:
driver.get(SERIE_A)
match_ids = []

In [9]:
year_selector = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/div/div/main/div[1]/div[2]/button')
year_selector.click()

div2022 = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/div/div/main/div[1]/div[2]/div/div[2]')
div2022.click()

time.sleep(1)

weeks = (
    driver.find_element(By.XPATH, '//*[@id="match-odds"]/div/div[1]/div[1]/div')
    ).find_elements(By.CSS_SELECTOR, "*")
next_week_btn = driver.find_element(By.XPATH, '//*[@id="match-odds"]/div/div[2]/button[2]')

In [17]:
for week in weeks:
    print(driver.find_element(By.XPATH, '//*[@id="match-odds"]/div/div[2]/div').text)
    games_in_current_page = driver.find_elements(By.CLASS_NAME, "match-url")
    for game in games_in_current_page:
        match_ids.append(game.get_attribute("href").split("=")[1])
    
    try:
        next_week_btn.click()
    except selenium.common.exceptions.ElementClickInterceptedException:
        pass

SEMANA 1
03 ABR - 09 ABR
SEMANA 2
10 ABR - 16 ABR
SEMANA 3
17 ABR - 23 ABR
SEMANA 4
24 ABR - 30 ABR
SEMANA 5
01 MAI - 07 MAI
SEMANA 6
08 MAI - 14 MAI
SEMANA 7
15 MAI - 21 MAI
SEMANA 8
22 MAI - 28 MAI
SEMANA 9
29 MAI - 04 JUN
SEMANA 10
05 JUN - 11 JUN
SEMANA 11
12 JUN - 18 JUN
SEMANA 12
19 JUN - 25 JUN
SEMANA 13
26 JUN - 02 JUL
SEMANA 14
03 JUL - 09 JUL
SEMANA 15
10 JUL - 16 JUL
SEMANA 16
17 JUL - 23 JUL
SEMANA 17
24 JUL - 30 JUL
SEMANA 18
31 JUL - 06 AGO
SEMANA 19
07 AGO - 13 AGO
SEMANA 20
14 AGO - 20 AGO
SEMANA 21
21 AGO - 27 AGO
SEMANA 22
28 AGO - 03 SET
SEMANA 23
04 SET - 10 SET
SEMANA 24
11 SET - 17 SET
SEMANA 25
18 SET - 24 SET
SEMANA 26
25 SET - 01 OUT
SEMANA 27
02 OUT - 08 OUT
SEMANA 28
09 OUT - 15 OUT
SEMANA 29
16 OUT - 22 OUT
SEMANA 30
23 OUT - 29 OUT
SEMANA 31
30 OUT - 05 NOV
SEMANA 32
06 NOV - 12 NOV


In [33]:
clean_data = {}
base_url = "https://oddspedia.com/api/v1/getMatchOdds?wettsteuer=0&geoCode=BR&bookmakerGeoCode=BR&bookmakerGeoState=&matchId={match_id}&oddGroupId={odds_market}&inplay=0&language=br".format
for match_id in match_ids:
    clean_data[match_id] = {}
    for market in [1, 3, 4, 12]:
        # ids {1: h2h, 3: spread, 4: over/under, 12: exact}
        driver.get(base_url(match_id = match_id, odds_market = market))
        match_odds = json.loads(driver.find_element(By.TAG_NAME, 'pre').text)
        for data in match_odds["data"]["prematch"]:
            if data["id"] == market:
                market_name = data["name"]
                clean_data[match_id][market_name] = []
                for period in data["periods"]:
                    if period["name"] == "Fim do Jogo":
                        for odd in data["periods"][0]["odds"]:
                            clean_data[match_id][market_name].append({
                                "SportsBook": odd["bookie_name"],
                                "odds_home": odd["o1"],
                                "odds_draw": odd["o2"],
                                "odds_away": odd["o3"],
                            })
                        break
                break
        break
    break

In [32]:
a

{'Resultado final': 1,
 'Total de Gols': 4,
 'Handicap Asiático': 3,
 'Ambas as Equipes Marcam': 11,
 'Chance Dupla': 7,
 'Empate Sem Aposta': 5,
 'Primeira Equipe a Marcar': 12,
 'Resultado Correto': 8,
 'Handicap Europeu': 6,
 'Handicap Asiático Cantos': 13,
 'Handicap Asiático Cartões': 16,
 'Total de cartões Mais/Menos': 19,
 'Intervalo/Fim do Jogo': 9,
 'Par ou Ímpar': 10,
 'Próximo Golo': 30,
 'Cantos par/ímpar': 18,
 'Baliza Inviolada': 21,
 'Para ganhar a zero': 23,
 'Ganhar De Trás': 24,
 'Para ganhar as duas metades': 25,
 'Marcar em ambas as metades': 27}